In [1]:
import itk
from itk import TubeTK as tube

import site

site.addsitedir("../../pytubetk")
from tube_visualization_utils import *
from tube_viewer import tube_viewer
from tube_utils import *

from segment_vessels_brain_mra import  segment_vessels_brain_mra

In [2]:
img = itk.imread("../data/MRI-Cases/mra-brain.mha", itk.F)
makeIso = tube.ResampleImage.New(img, MakeHighResIso=True)
makeIso.Update()
img_iso = makeIso.GetOutput()

img_org = itk.imread("../data/MRI-Cases/mra.mha", itk.F)
makeIso = tube.ResampleImage.New(img_org, MakeHighResIso=True)
makeIso.Update()
img_org_iso = makeIso.GetOutput()

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


In [3]:
vess = segment_vessels_brain_mra(debug=True, debug_output_base_name="Debug/brain")
vess.set_input_image(img_iso)

initializing image
Saving mask image


In [4]:
scales = [1.00]
vess_out = []
for scale in scales:
    vess.normalize_image(vessels_scale=scale)
    vess.identify_training_seeds()
    vess.generate_training_mask_image()
    vess.enhance_vessels()
    vess.extract_vessels(alter_image=img_org_iso) 
    vess_out.append(vess.data_vessels_group)

Normalizing image
data stats = 82.136536 18.571918
data = 0.0 0.67930484
Saving normalized image
Identifying training seeds
[ -1.82006   -4.825642 -28.399886] : 98.300156
[  9.097912 -15.743614 -25.460432] : 98.04999
[-13.157954 -16.163536 -25.880354] : 97.111015
[ -3.079826 -30.440884 -12.44285 ] : 91.89418
[ -2.239982 -40.09909   -1.524878] : 91.64016
[ -1.400138 -25.40182  -17.901836] : 89.96093
[ -7.698968 -42.618622  11.072782] : 89.404366
[ -4.339592 -31.280728  -8.24363 ] : 89.34921
[ -3.079826  -6.925252 -33.019028] : 89.09401
[-24.075926 -22.042444 -23.360822] : 88.53641
[ -3.91967  -54.79636   12.332548] : 88.26187
[ -3.079826 -35.479948  -4.04441 ] : 88.16008
[-11.898188 -27.50143  -36.378404] : 87.54731
[-19.876706 -17.843224 -23.360822] : 87.471
[ 27.57448  -22.462366 -21.681134] : 86.19409
[ 15.816664 -16.163536 -21.681134] : 86.05617
[  8.258068 -14.90377  -35.958482] : 84.402176
[ 11.617444 -19.522912 -35.118638] : 83.85529
[ -6.859124  -4.825642 -27.14012 ] : 83.148254

In [6]:
viewer1 = tube_viewer(tubes=vess_out[0])
viewer1.render_tubes_as_surfaces()

Selected position:  [19.951378556688194, -37.802465252690354, -29.02788878503381]
Selected position:  [0.22804242738661173, -44.29045206127374, -17.721387976274997]
Selected position:  [-16.0441253013936, -15.77567750869838, -23.556015204135434]
Selected position:  [-8.01627861763788, 12.885346870975589, 20.56153503207301]
Selected position:  [-9.890485598223165, -0.35773761990677144, 26.958045409271037]
Selected position:  [-8.013796849266726, -30.321472899526228, 24.225217936271974]


In [ ]:
vgrp_base = read_group("./brain.tre")
viewer2 = tube_viewer(tubes=vgrp_base)
viewer2.render_tubes_as_surfaces(param="Radius")